In [159]:
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [160]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

In [161]:
%%R

if (require("sandwich") == FALSE) {
  install.packages("sandwich")
  library(sandwich)
}

Loading required package: sandwich


In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

In [313]:
hub1_name = "ttf"
hub2_name = "the"
horizon = 250
window_size = 5
model = "tvecm"

data = f"{hub1_name}_{hub2_name}_h{horizon}_w{window_size}"


predictions = pd.read_csv(f"../predictions/{data}_{model}_predictions.csv")
last_available_data = pd.read_csv(f"../predictions/{data}_last_available_data.csv")
actuals = pd.read_csv(f"../predictions/{data}_actuals.csv")

In [314]:
predictions = predictions.set_index("Date")
last_available_data = last_available_data.set_index("Date")
actuals = actuals.set_index("Date")

In [315]:


hub1_predictions = predictions[[hub1_name]]
hub1_last_available_data = last_available_data[[hub1_name]]
hub1_actuals = actuals[[hub1_name]]

hub1_predicted_returns = np.log(hub1_predictions / hub1_last_available_data)*100

hub2_predictions = predictions[[hub2_name]]
hub2_last_available_data = last_available_data[[hub2_name]]
hub2_actuals = actuals[[hub2_name]]

hub2_predicted_returns = np.log(hub2_predictions / hub2_last_available_data)*100


In [316]:
fig = px.line(
    x=hub1_predicted_returns.index,
    y=[hub1_predicted_returns[hub1_name], hub2_predicted_returns[hub2_name]],
    labels={'x': 'Date', 'value': 'Predicted Returns', 'variable': 'Hub'},
    title='Predicted Returns for Hub1 and Hub2'
)

fig.update_layout(
    legend_title_text='Hub',
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

fig.show()

In [317]:
forecast_horizon = len(hub1_predicted_returns)

In [318]:
def trading_system(forecast_horizon, hub1_predicted_returns, hub2_predicted_returns, hub1_last_available_data, hub2_last_available_data, hub1_actuals, hub2_actuals, threshold=0.5):
    

    transaction_costs = 0.005

    trades = [0] * forecast_horizon
    pl = [0] * forecast_horizon
    returns = [0] * forecast_horizon
    returns_with_transaction_costs = [0] * forecast_horizon

    profit = 0

    for i in range(forecast_horizon):
        hub1_predicted_return = hub1_predicted_returns.values[i].item()
        hub2_predicted_return = hub2_predicted_returns.values[i].item()

        if hub1_predicted_return > hub2_predicted_return + threshold:

            long_position_return = (hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item()) / hub1_last_available_data.values[i].item()
            short_position_return = (hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item()) / hub2_last_available_data.values[i].item()

            long_position_return_with_transaction_costs = (hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item() - 2*transaction_costs) / hub1_last_available_data.values[i].item()
            short_position_return_with_transaction_costs = (hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item() - 2*transaction_costs) / hub2_last_available_data.values[i].item()

            transaction_cost_entry = transaction_costs * 2
            net_profit_entry = hub2_last_available_data.values[i].item() - hub1_last_available_data.values[i].item() - transaction_cost_entry

            transaction_cost_exit = transaction_costs * 2
            net_profit_exit = hub1_actuals.values[i].item() - hub2_actuals.values[i].item() - transaction_cost_exit

            profit += (net_profit_entry + net_profit_exit)

            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        elif hub2_predicted_return > hub1_predicted_return + threshold:

            long_position_return = (hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item()) / hub2_last_available_data.values[i].item()
            short_position_return = (hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item()) / hub1_last_available_data.values[i].item()

            long_position_return_with_transaction_costs = (hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item() - 2*transaction_costs) / hub2_last_available_data.values[i].item()
            short_position_return_with_transaction_costs = (hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item() - 2*transaction_costs) / hub1_last_available_data.values[i].item()

            transaction_cost_entry = transaction_costs * 2
            net_profit_entry = hub1_last_available_data.values[i].item() - hub2_last_available_data.values[i].item() - transaction_cost_entry

            transaction_cost_exit = transaction_costs * 2
            net_profit_exit = hub2_actuals.values[i].item() - hub1_actuals.values[i].item() - transaction_cost_exit

            profit += (net_profit_entry + net_profit_exit) 
            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        else:
            pass

        pl[i] = profit

    return pl, profit, trades, returns, returns_with_transaction_costs

In [328]:
pl, profit, trades, returns, returns_with_transaction_costs = trading_system(
    forecast_horizon, 
    hub1_predicted_returns, hub2_predicted_returns, 
    hub1_last_available_data, hub2_last_available_data, 
    hub1_actuals, hub2_actuals, 
    threshold=2)

print(f"Profit: {profit}")

Profit: 30.456000000000003


In [329]:
print(np.mean(returns))
print(np.mean(returns_with_transaction_costs))

0.004536987020362501
0.004486814824231158


In [330]:
standard_error_returns = np.std(returns) / np.sqrt(len(returns))
(np.mean(returns) - 1.96 * standard_error_returns)*100, (np.mean(returns) + 1.96 * standard_error_returns)*100

(np.float64(0.15749423463049889), np.float64(0.7499031694420013))

In [331]:
standard_error_returns_with_transaction_costs = np.std(returns_with_transaction_costs) / np.sqrt(len(returns_with_transaction_costs))
(np.mean(returns) - 1.96 * standard_error_returns_with_transaction_costs)*100, (np.mean(returns) + 1.96 * standard_error_returns_with_transaction_costs)*100

(np.float64(0.1585652272093849), np.float64(0.7488321768631153))

In [323]:
# Calculate win and loss rates for returns
win_rate_returns = sum(1 for r in returns if r > 0) / len(returns)
loss_rate_returns = sum(1 for r in returns if r <= 0) / len(returns)

# Calculate win and loss rates for returns with transaction costs
win_rate_returns_with_tc = sum(1 for r in returns_with_transaction_costs if r > 0) / len(returns_with_transaction_costs)
loss_rate_returns_with_tc = sum(1 for r in returns_with_transaction_costs if r <= 0) / len(returns_with_transaction_costs)

print(f"Win rate for returns: {win_rate_returns:.2%}")
print(f"Loss rate for returns: {loss_rate_returns:.2%}")
print(f"Win rate for returns with transaction costs: {win_rate_returns_with_tc:.2%}")
print(f"Loss rate for returns with transaction costs: {loss_rate_returns_with_tc:.2%}")

Win rate for returns: 22.40%
Loss rate for returns: 77.60%
Win rate for returns with transaction costs: 22.00%
Loss rate for returns with transaction costs: 78.00%


In [324]:
fig = go.Figure()

# Add line plot for pl
fig.add_trace(go.Scatter(
    x=list(range(forecast_horizon)),
    y=pl,
    mode='lines',
    name='Profit/Loss'
))

# Add scatter plot for trades
fig.add_trace(go.Scatter(
    x=list(range(forecast_horizon)),
    y=trades,
    mode='markers',
    name='Trades',
    yaxis='y2'
))

# Create axis objects
fig.update_layout(
    title='Profit/Loss and Trades Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Profit/Loss'),
    yaxis2=dict(title='Trades', overlaying='y', side='right')
)

fig.show()

In [325]:
fig = px.line(
    actuals,
    x=actuals.index,
    y=[hub1_name, hub2_name],
    labels={'x': 'Date', 'value': 'Actual Values', 'variable': 'Hub'},
    title='Actual Values for Hub1 and Hub2'
)

fig.update_layout(
    legend_title_text='Hub',
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

fig.show()

In [326]:
np.random.seed(42)
profit = 0

for i in range(forecast_horizon):
    hub1_predicted_return = hub1_predicted_returns.values[i].item()
    hub2_predicted_return = hub2_predicted_returns.values[i].item()


    if np.random.rand() > 1:
        print(f"Buy {hub1_name} and sell {hub2_name}")
        net_profit_entry = hub2_last_available_data.values[i].item() - hub1_last_available_data.values[i].item()
        net_profit_exit = hub1_actuals.values[i].item() - hub2_actuals.values[i].item()
        profit += net_profit_entry + net_profit_exit

    else:
        print(f"Buy {hub2_name} and sell {hub1_name}")
        net_profit_entry = hub1_last_available_data.values[i].item() - hub2_last_available_data.values[i].item()
        net_profit_exit = hub2_actuals.values[i].item() - hub1_actuals.values[i].item()
        profit += net_profit_entry + net_profit_exit
        
print(f"Profit: {profit}")

Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and sell ttf
Buy the and s

In [327]:
hub1_predictions.values[0].item() > hub2_predictions.values[0].item()

False